# End-to-end test -- from a DSM to a DTM
with error/bias analysis

## Imports
### pytorch-unet

In [1]:
import os

if not os.path.exists("pytorch_unet.py"):
    if not os.path.exists("pytorch_unet"):
        !git clone https://github.com/akigoat/pytorch-unet.git

    # %cd pytorch-unet

fatal: destination path 'pytorch-unet' already exists and is not an empty directory.


In [7]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm, trange
import math
# files
import requests
from io import BytesIO
from zipfile import ZipFile
import os
import shutil
from pathlib import Path
# from scipy import interpolate
import matplotlib.pyplot as plt
# GIS
from osgeo import gdal
import rasterio
from affine import Affine
import richdem as rd
from rasterio.fill import fillnodata
# torch and dataset
import torch
import torchvision
from torch.utils.data import Dataset

# functions
import sys
sys.path.append('pytorch-unet')
import helper
import loss
import simulation

import dataFunctions
import netClasses

In [5]:
print(torch.backends.mps.is_available())
print(torch.backends.mps.is_built())

True
True


In [10]:
device = torch.device("mps")
path_folder = '/Volumes/HydesT7/Grad Project/Data/U_Net/64_64_1_8/'
path_model_folder = '/Volumes/HydesT7/Grad Project/Model/U_Net/1013/'
path_ANH4 = '/Volumes/HydesT7/Grad Project/Data/AHN4_all/'

In [15]:
model = netClasses.ResNetUNet(2)
model = model.to(device)

model.load_state_dict(torch.load(path_model_folder+'roughness_20_1e-4.pt'))
model.eval()

ResNetUNet(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tra

In [16]:
list_patch = []
with open(path_ANH4+'picked.txt') as fp:
    # list_patch.append(patch)
    list_patch = [line.rstrip('\n') for line in fp]
print(list_patch[:5], len(list_patch))

['05FZ1', '05FZ2', '05GN2', '05GZ1', '05GZ2'] 505


In [18]:
from pathlib import Path
Path("data/tmp").mkdir(parents=True, exist_ok=True)
tmp_path = 'data/tmp/'